In [24]:
from fastembed import TextEmbedding
import numpy as np

### Q1. Embedding the query

In [6]:
model_name = "jinaai/jina-embeddings-v2-small-en"

In [45]:
query = "I just discovered the course. Can I join now?"

In [46]:
model = TextEmbedding(model_name=model_name)
embeddings = list(model.embed(query))[0]

In [26]:
len(embeddings)

512

In [51]:
# embedding

In [30]:
min_value = np.min(embeddings)
min_value

-0.11726373885183883

In [31]:
np.linalg.norm(embeddings)

1.0

### Q2. Cosine similarity with another vector

In [32]:
doc = 'Can I still join the course after the start date?'
doc_embeddings = list(model.embed(doc))[0]

In [34]:
doc_embeddings.dot(embeddings)

0.9008528895674548

In [35]:
embeddings.dot(doc_embeddings)

0.9008528895674548

### Q3. Ranking by cosine

In [37]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [59]:
# Extract text fields from documents
doc_texts = [doc['text'] for doc in documents]
doc_texts

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [61]:
# Compute embeddings for documents and query
doc_embeddings = np.array(list(model.embed(doc_texts)))  # Shape: (n_documents, embedding_dim)
query_embedding = np.array(list(model.embed([query]))[0])  # Shape: (embedding_dim,)

doc_embeddings.shape, query_embedding.shape


((5, 512), (512,))

In [64]:
# Normalize embeddings for cosine similarity
doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)
query_embedding = query_embedding / np.linalg.norm(query_embedding)



In [65]:
# Compute cosine similarity (V.dot(q))
cosine_similarities = doc_embeddings.dot(query_embedding)
cosine_similarities

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [66]:
# Find the index of the document with the highest similarity
max_similarity_index = np.argmax(cosine_similarities)
max_similarity_index

1

In [72]:
import json

print("Corresponding document text:", json.dumps(documents[max_similarity_index], indent=3))

Corresponding document text: {
   "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
   "section": "General course-related questions",
   "question": "Course - Can I follow the course after it finishes?",
   "course": "data-engineering-zoomcamp"
}


### Q4. Ranking by cosine, version two

In [73]:
full_doc_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_doc_texts

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [74]:
full_text_doc_embeddings = np.array(list(model.embed(full_doc_texts)))  # Shape: (n_documents, embedding_dim)

In [75]:
full_text_doc_embeddings = full_text_doc_embeddings / np.linalg.norm(full_text_doc_embeddings, axis=1, keepdims=True)

In [76]:
full_text_cosine_similarities = full_text_doc_embeddings.dot(query_embedding)
full_text_cosine_similarities

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [77]:
# Find the index of the document with the highest similarity
max_similarity_index = np.argmax(full_text_cosine_similarities)
max_similarity_index

0

### Q5. Selecting the embedding model

In [78]:
EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] <= EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    

### Q6. Indexing with qdrant (2 points)

In [79]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [81]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [86]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")

small_model_name = "BAAI/bge-small-en"
# Define the collection name
collection_name = "zoomcamp-home-work"

client.delete_collection(collection_name=collection_name)

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [87]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=small_model_name)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [89]:
from tqdm import tqdm

def upsert_with_progress(client, collection_name, points, batch_size=100):
    # Convert points to list if it isn't already
    if not isinstance(points, list):
        points = list(points)
    
    # Process in batches with progress bar
    for i in tqdm(range(0, len(points), batch_size), desc="Uploading batches"):
        batch = points[i:i + batch_size]
        client.upsert(
            collection_name=collection_name,
            points=batch
        )
    
    print(f"✅ Successfully upserted {len(points)} points")

# Usage
upsert_with_progress(client, collection_name, points, batch_size=50)

Uploading batches:   0%|                                                                                                                                   | 0/8 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Uploading batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:51<00:00, 13.89s/it]

✅ Successfully upserted 375 points


In [90]:
def search(query, limit=5):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=small_model_name
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [95]:
query
result = search(query)
result.points[0].score

0.8703172